<a href="https://colab.research.google.com/github/Miriyamsreevaishnavi/learnopencv/blob/master/yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [3]:
!nvidia-smi

Wed Mar  5 16:38:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   67C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
!git clone https://github.com/ThensiB/RepVGG-GELAN.git

Cloning into 'RepVGG-GELAN'...
remote: Enumerating objects: 193, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 193 (delta 14), reused 7 (delta 7), pack-reused 171 (from 1)
Receiving objects: 100% (193/193), 899.23 KiB | 2.60 MiB/s, done.
Resolving deltas: 100% (59/59), done.


In [6]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 53.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

# k cross validation

In [7]:
import os
import yaml
import shutil

# Define paths
yolov9_path = '/content/RepVGG-GELAN'  # Update with the path to YOLOv9 directory
custom_yaml_path = '/content/RepVGG-GELAN/data/coco.yaml'  # Update with the path to custom.yaml
data_path = '/content/drive/MyDrive/Folds'


In [8]:
# Function to create training folders for each fold
def create_train_folders(train_folders, n):
    for idx, folder in enumerate(train_folders):
        train_folder = f'/content/drive/MyDrive/Folds/train{n}'
        os.makedirs(os.path.join(train_folder, 'images'), exist_ok=True)
        os.makedirs(os.path.join(train_folder, 'labels'), exist_ok=True)

        # Copy images and labels from original training folders
        images_src = os.path.join(folder, 'images')
        labels_src = os.path.join(folder, 'labels')

        print(labels_src)

        images_dst = os.path.join(train_folder, 'images')
        labels_dst = os.path.join(train_folder, 'labels')

        print(images_dst)

        for image_file in os.listdir(images_src):
            shutil.copy(os.path.join(images_src, image_file), images_dst)

        for label_file in os.listdir(labels_src):
            shutil.copy(os.path.join(labels_src, label_file), labels_dst)

        # Count images and labels
        num_images = len(os.listdir(images_dst))
        num_labels = len(os.listdir(labels_dst))
    print(f"Train{n} - Images: {num_images}, Labels: {num_labels}")

    return train_folder


In [9]:
import shutil
p = '/content/drive/MyDrive/Folds/train1'
shutil.rmtree(p)

In [11]:
# Function to train YOLOv9
def train_yolov9(train_folders, val_folder, fold_idx):
    # Load custom.yaml
    with open(custom_yaml_path, 'r') as file:
        yaml_content = yaml.safe_load(file)

    # Update train and val paths
    yaml_content['path'] = data_path
    yaml_content['train'] = train_folders
    yaml_content['val'] = val_folder

    print(yaml_content['val'])

    # Write the updated YAML content back to the file
    with open(custom_yaml_path, 'w') as file:
        yaml.dump(yaml_content, file)

    !python {yolov9_path}/train.py --img 640 --batch 8 --epochs 5 --min-items 0 --data {custom_yaml_path} --cfg {yolov9_path}/models/detect/rcs-gelan-c.yaml --hyp {yolov9_path}/data/hyps/hyp.scratch-high.yaml --device 0 --name fold{fold_idx}


# Get list of fold directories
fold_directories = [f'/content/drive/MyDrive/Folds/fold_{i+1}' for i in range(5)]

# Iterate over each fold for validation
for fold_idx, val_folder in enumerate(fold_directories):

    if fold_idx == 4:
      print(f"Training on fold {fold_idx + 1}...")

      # Define training folders (all other folds)
      train_folders = fold_directories[:fold_idx] + fold_directories[fold_idx + 1:]

      # Create training folders for the current fold
      train_fold = create_train_folders(train_folders, fold_idx + 1)
      print(train_fold)

      # Train YOLOv9 on the current fold
      train_yolov9(train_fold, val_folder, fold_idx + 1)



Training on fold 5...
/content/drive/MyDrive/Folds/fold_1/labels
/content/drive/MyDrive/Folds/train5/images
/content/drive/MyDrive/Folds/fold_2/labels
/content/drive/MyDrive/Folds/train5/images
/content/drive/MyDrive/Folds/fold_3/labels
/content/drive/MyDrive/Folds/train5/images
/content/drive/MyDrive/Folds/fold_4/labels
/content/drive/MyDrive/Folds/train5/images
Train5 - Images: 400, Labels: 400
/content/drive/MyDrive/Folds/train5
/content/drive/MyDrive/Folds/fold_5
2025-03-05 16:49:00.382720: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741193340.621536    4376 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741193340.690923    4376 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS wh

In [13]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: vtu20117 (datalo) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [14]:
#RCS-GELAN

# Function to train YOLOv9
def train_yolov9(train_folders, val_folder, fold_idx):
    # Load custom.yaml
    with open(custom_yaml_path, 'r') as file:
        yaml_content = yaml.safe_load(file)

    # Update train and val paths
    yaml_content['path'] = data_path
    yaml_content['train'] = train_folders
    yaml_content['val'] = val_folder

    print(yaml_content['val'])

    # Write the updated YAML content back to the file
    with open(custom_yaml_path, 'w') as file:
        yaml.dump(yaml_content, file)

    !python {yolov9_path}/train.py --img 640 --batch 8 --epochs 5 --min-items 0 --close-mosaic 15 --data {custom_yaml_path} --cfg {yolov9_path}/models/detect/rcs-gelan-c.yaml --hyp {yolov9_path}/data/hyps/hyp.scratch-high.yaml --device 0 --name fold{fold_idx}


# Get list of fold directories
fold_directories = [f'/content/drive/MyDrive/Folds/fold_{i+1}' for i in range(5)]

# Iterate over each fold for validation
for fold_idx, val_folder in enumerate(fold_directories):
    print(f"Training on fold {fold_idx + 1}...")

    # Define training folders (all other folds)
    train_folders = fold_directories[:fold_idx] + fold_directories[fold_idx + 1:]

    # Create training folders for the current fold
    train_fold = create_train_folders(train_folders, fold_idx + 1)
    print(train_fold)

    # Train YOLOv9 on the current fold
    train_yolov9(train_fold, val_folder, fold_idx + 1)

# After training, you can aggregate and analyze the results as needed


Training on fold 1...
/content/drive/MyDrive/Folds/fold_2/labels
/content/drive/MyDrive/Folds/train1/images
/content/drive/MyDrive/Folds/fold_3/labels
/content/drive/MyDrive/Folds/train1/images
/content/drive/MyDrive/Folds/fold_4/labels
/content/drive/MyDrive/Folds/train1/images
/content/drive/MyDrive/Folds/fold_5/labels
/content/drive/MyDrive/Folds/train1/images
Train1 - Images: 400, Labels: 400
/content/drive/MyDrive/Folds/train1
/content/drive/MyDrive/Folds/fold_1
2025-03-05 16:53:21.245062: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741193601.278459    6034 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741193601.288742    6034 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS wh

# training on brain dataset

In [15]:
!python /content/RepVGG-GELAN/train.py \
--batch 8 --epochs 5 --img 640 --device 0 --min-items 0 --close-mosaic 15 \
--data /content/RepVGG-GELAN/data/coco.yaml \
--cfg /content/RepVGG-GELAN/models/detect/rcs-gelan-c.yaml \
--hyp /content/RepVGG-GELAN/data/hyps/hyp.scratch-high.yaml

2025-03-05 17:25:12.653018: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741195512.686553   14809 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741195512.696421   14809 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-05 17:25:12.728233: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb:

In [16]:
!python /content/RepVGG-GELAN/train.py \
--batch 8 --epochs 5 --img 640 --device 0 --min-items 0 --close-mosaic 15 \
--data /content/RepVGG-GELAN/data/coco.yaml \
--cfg /content/RepVGG-GELAN/models/detect/rcs-gelan-c.yaml \
--hyp /content/RepVGG-GELAN/data/hyps/hyp.scratch-high.yaml

2025-03-05 17:28:38.176424: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741195718.197344   15767 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741195718.203758   15767 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-05 17:28:38.225092: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb:

In [17]:
!python /content/RepVGG-GELAN/train.py \
--batch 8 --epochs 5 --img 640 --device 0 --min-items 0 --close-mosaic 15 \
--data /content/RepVGG-GELAN/data/coco.yaml \
--cfg /content/RepVGG-GELAN/models/detect/rcs-gelan-c.yaml \
--hyp /content/RepVGG-GELAN/data/hyps/hyp.scratch-high.yaml

2025-03-05 17:32:09.145655: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741195929.166492   16742 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741195929.172671   16742 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-05 17:32:09.193172: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb:

In [18]:
!python /content/RepVGG-GELAN/train.py \
--batch 8 --epochs 5 --img 640 --device 0 --min-items 0 --close-mosaic 15 \
--data /content/RepVGG-GELAN/data/coco.yaml \
--cfg /content/RepVGG-GELAN/models/detect/rcs-gelan-c.yaml \
--hyp /content/RepVGG-GELAN/data/hyps/hyp.scratch-high.yaml

2025-03-05 17:35:50.654093: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741196150.674787   17772 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741196150.681014   17772 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-05 17:35:50.701968: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb:

# yolov8 on brain

In [19]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.2/922.2 kB 32.1 MB/s eta 0:00:00


In [20]:
from ultralytics import YOLO

from IPython.display import display, Image

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [21]:
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

In [22]:
ROOT_DIR = '/content/drive/MyDrive'

In [23]:
# Load a model
model = YOLO("yolov8n.pt")  # load pre trained model

# Use the model
results = model.train(data=os.path.join(ROOT_DIR, "coco.yaml"), epochs=150)  # train the model


100%|██████████| 6.25M/6.25M [00:00<00:00, 83.8MB/s]


Ultralytics 8.3.84 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/coco.yaml, epochs=150, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, sho

100%|██████████| 5.35M/5.35M [00:00<00:00, 101MB/s]


AMP: checks passed ✅


train: Scanning /content/drive/MyDrive/Folds/train5/labels... 400 images, 3 backgrounds, 0 corrupt: 100%|██████████| 400/400 [00:04<00:00, 98.29it/s] 


...

    150/150      1.99G     0.4981     0.2866     0.8939         16        640: 100%|██████████| 25/25 [00:09<00:00,  2.52it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.61it/s]

                   all        100        114      0.954      0.916      0.983      0.768



150 epochs completed in 0.445 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.84 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.73it/s]


                   all        100        114      0.945      0.939      0.983       0.78
Speed: 0.3ms preprocess, 3.0ms inference, 0.0ms loss, 3.7ms postprocess per image
Results saved to runs/detect/train


In [25]:
!python /content/RepVGG-GELAN/train.py \
--batch 8 --epochs 5 --img 640 --device 0 --min-items 0 --close-mosaic 15 \
--data /content/RepVGG-GELAN/data/coco.yaml \
--cfg /content/RepVGG-GELAN/models/detect/rcs-gelan-c.yaml \
--hyp /content/RepVGG-GELAN/data/hyps/hyp.scratch-high.yaml

E0000 00:00:1741198107.594636   26298 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741198107.600967   26298 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: vtu20117 (datalo) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
train: weights=, cfg=/content/RepVGG-GELAN/models/detect/rcs-gelan-c.yaml, data=/content/RepVGG-GELAN/data/coco.yaml, hyp=/content/RepVGG-GELAN/data/hyps/hyp.scratch-high.yaml, epochs=5, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=0, multi_scale=False, single_cls=False, optimizer=SGD, 